In [3]:
import os
import pandas as pd
from pandas.compat import StringIO
import numpy as np
from numpy import loadtxt
import sys
import json
from pprint import pprint
import objectpath
import csv
import re
import matplotlib.pyplot as plt
import json
import io
import http.client
import requests
from pprint import pprint
import itertools
import scipy as sp
from scipy.spatial import distance
from sklearn.metrics.pairwise import pairwise_distances
from flask import Flask, render_template, request, redirect, jsonify, send_from_directory, abort, Response, send_file
import urllib.request
import h5py
import h5sparse
from scipy.sparse import csc_matrix, csr_matrix, coo_matrix, vstack
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import keras
from keras import backend as K
import tensorflow
from clustergrammer2 import net, Network
from clustergrammer_widget import *
from gvm_scripts import *
from vae_scripts import * 

GWAS_cat = {}

for line in urllib.request.urlopen('http://amp.pharm.mssm.edu/lincs-playground/index.php/s/9Y78KySXTy4IqOc/download'):
    label, genelist = line.decode().split('\t', maxsplit=1) 
    genelist_split = genelist.strip().split('\t')
    GWAS_cat[label] = genelist_split

GWAS_phenotypes_list = list(GWAS_cat.keys())
GWAS_cat = {k.replace('_', ' '): v for k, v in GWAS_cat.items()}
#GWAS_drug_and_disease_list
GWAS_phenotypes_list_no_spaces = []
for i in GWAS_phenotypes_list:
    j = i.replace('_',' ')
    GWAS_phenotypes_list_no_spaces.append(j)
#GWAS_drug_and_disease_list

user_input = "alcohol" ## CHANGE THIS TO WHATEVER THE USER INPUT IS (EITHER DISEASE OR DRUGS)

GWAS_cat_dict ={
    k: v
    for k, v in GWAS_cat.items()
    if re.compile(user_input).search(k)
}

GWAS_cat_df = pd.DataFrame(GWAS_cat.items())
GWAS_cat_df.columns = ["Phenotype", "Genes"]

GWAS_input_subset = GWAS_cat_df[GWAS_cat_df["Phenotype"].apply(
    lambda s: bool(re.compile(str(user_input), re.IGNORECASE).search(str(s))))] # can display this dataframe (it won't look great. Maybe just display the first column?)
GWAS_phenotype_df = pd.DataFrame(GWAS_input_subset.Phenotype) #display these names!


if len(GWAS_cat_dict) > 0:
    GWAS_json = json.dumps(GWAS_cat_dict)
    GWAS_filename = open(user_input + "_GWAS.json", "w") 
    GWAS_filename.write(GWAS_json) # saves the json 
    GWAS_filename.close()
    print("Significant genes for " + user_input + " found in GWAS")
else:
    print("No significant genes for " + user_input + " found in GWAS")

CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
CREEEDS_Drug_response = requests.get(CREEDS_URL + 'search', params={'q':"ethanol"})
if CREEEDS_Drug_response.status_code == 200:
    #pprint(CREEEDS_Drug_response.json())
    #json.dump(CREEEDS_Drug_response.json(), open(DrOI + '_api1_result.json', 'w'), indent=4)
    CREEDS_drug_output_df = pd.DataFrame(CREEEDS_Drug_response.json())
    CREEDS_drug_output_ids = list(CREEDS_drug_output_df["id"])

    CREEDS_drug_output_ids_up = ["CREEDS_" + s + "_up" for s in CREEDS_drug_output_ids]
    CREEDS_drug_output_ids_down = ["CREEDS_" + s + "_down" for s in CREEDS_drug_output_ids]

    CREEDS_all_down_genes = []
    CREEDS_all_up_genes = []
    CREEDS_desc = []
    CREEDS_data = []
    for a in CREEDS_drug_output_ids:
        CREEDS_URL = 'http://amp.pharm.mssm.edu/CREEDS/'
        CREEDS_drug_sigs_response = requests.get(CREEDS_URL + 'api', params={'id':str(a)})
        if CREEDS_drug_sigs_response.status_code == 200:
            CREEDS_drug_sigs_response_json = CREEDS_drug_sigs_response.json()

            ## up genes
            CREEDS_drug_sigs_up_genes = CREEDS_drug_sigs_response_json['up_genes']
            CREEDS_len_up = len(CREEDS_drug_sigs_up_genes)
            
            CREEDS_drug_sigs_up_genes_df = pd.DataFrame(CREEDS_drug_sigs_up_genes) # this is the up genes dataframe
            CREEDS_drug_sigs_up_genes_df.columns = ["Genes", "Score"]
            filename1 = (a + "_CREEDS_drug_sig_up_genes.csv")
            #CREEDS_drug_sigs_up_genes_df.to_csv(filename1) # this saves the df as a csv
            desc = (a + "_" + "ethanol" + "_" + CREEDS_drug_sigs_response_json["geo_id"])
            CREEDS_desc.append(desc)
            CREEDS_all_up_genes.append(list(CREEDS_drug_sigs_up_genes_df["Genes"]))

            ## down genes
            CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes']
            CREEDS_len_down = len(CREEDS_drug_sigs_down_genes)
            CREEDS_drug_sigs_down_genes_df = pd.DataFrame(CREEDS_drug_sigs_down_genes)# this is the down genes dataframe
            CREEDS_drug_sigs_down_genes_df.columns = ["Genes", "Score"]
            filename2 = (a + "_CREEDS_drug_sig_down_genes.csv")
            CREEDS_all_down_genes.append(list(CREEDS_drug_sigs_down_genes_df["Genes"]))
            #CREEDS_drug_sigs_down_genes_df.to_csv(filename2)
            #CREEDS_drug_sigs_down_genes = CREEDS_drug_sigs_response_json['down_genes'] # this saves the df as a csv
            #print(filename2)
            CREEDS_data.append([a, CREEDS_len_up, CREEDS_len_down])
            up_keys = ['up_genes']
            gene_dict_up = {x:CREEDS_drug_sigs_response_json[x] for x in up_keys}
            gene_dict_up = {"CREEDS_" + a + "_" + k: v for k, v in gene_dict_up.items()}
           
            down_keys = ['down_genes']
            gene_dict_down = {x:CREEDS_drug_sigs_response_json[x] for x in down_keys}
            gene_dict_down = {"CREEDS_" + a + "_" + k: v for k, v in gene_dict_down.items()}
            
CREEDS_df = pd.DataFrame(CREEDS_data, columns=["Signature", "Number of Up Genes", "Number of Down Genes"])
print(CREEDS_df)
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z
# DrOI_up_no_perts_cp = {"L1000FWD_" + k +"_up": v for k, v in DrOI_up_no_perts.items()}
# DrOI_down_no_perts_cp = {"L1000FWD_" + k +"_down": v for k, v in DrOI_down_no_perts.items()}
# ## Genes
# DrugMatrix_Drug_Genes = merge_two_dicts(DrugMatrix_up_sigs , DrugMatrix_down_sigs)
# DrugMatrix_Drug_Genes = {"DrugMatrix_" + k: v for k, v in DrugMatrix_Drug_Genes.items()}
# L1000_Drug_Genes = merge_two_dicts(DrOI_up_no_perts_cp,DrOI_down_no_perts_cp)
CREEDS_up_Genes = {
    CREEDS_drug_output_ids_up[a]: CREEDS_all_up_genes[a]
    for a in range(len(CREEDS_drug_output_ids_up))
}


CREEDS_down_Genes = {
    CREEDS_drug_output_ids_down[a]: CREEDS_all_down_genes[a]
    for a in range(len(CREEDS_drug_output_ids_up))
}


# CREEDS_Disease_up_Genes = {
#     CREEDS_disease_output_ids_up[a]: CREEDS_all_disease_up_genes[a]
#     for a in range(len(CREEDS_drug_output_ids_up))
# }
# CREEDS_Disease_down_Genes = {
#     CREEDS_disease_output_ids_down[a]: CREEDS_all_disease_down_genes[a]
#     for a in range(len(CREEDS_drug_output_ids_up))
# }
# total_genes = merge_two_dicts(L1000_Drug_Genes, DrugMatrix_Drug_Genes)
# total_genes =merge_two_dicts(total_genes, geneshot_coexp_ass)
# total_genes =merge_two_dicts(total_genes, geneshot_generif)
# total_genes =merge_two_dicts(total_genes, geneshot_tagger)
# total_genes =merge_two_dicts(total_genes, geneshot_autorif)
# total_genes =merge_two_dicts(total_genes, geneshot_enrichr)
total_genes =merge_two_dicts(CREEDS_up_Genes, CREEDS_down_Genes)
print(len(CREEDS_up_Genes))
print(len(CREEDS_down_Genes))
total_genes =merge_two_dicts(total_genes, GWAS_cat_dict)
print(len(GWAS_cat_dict))
# total_genes =merge_two_dicts(total_genes, CREEDS_Disease_up_Genes)
# total_genes =merge_two_dicts(total_genes, CREEDS_Disease_down_Genes)
## you will need to change this path file. 
print("./data/" + "ethanol")
with open ("./data/" + "ethanol_merged"+".gmt", "w") as file:
    for k in list(total_genes.keys()):
        file.write(k + '\t')
        #file.write('\t'+'na')
        file.write("\t".join(total_genes[k]))
        file.write('\n')



#AUTO ENCODER CODE



all_genes = pd.read_csv('./data/AE/genes_info.tsv', sep='\t', index_col=0)
gmt_fname = "./data/" + "ethanol_merged" + '.gmt'
lib_name = os.path.splitext(gmt_fname.rsplit('/', 1)[-1])[0]
gvm_fname = './data/' + lib_name + '.h5'
print(gvm_fname)
formatted_gvm_fname = './data/' + lib_name + '_FORMATTED.h5'

if os.path.isfile(gvm_fname): 
    gvm = open_gvm(gvm_fname)
else:
    gvm = convert_genesetlist(get_genesetlist(gmt_fname, 'gmt_fname'), to='gvm_h5', output_fname=gvm_fname)

summary = format_gvm_h5(gvm_fname = gvm_fname, all_genes = all_genes,
                   output_fname = formatted_gvm_fname, max_gs_loss=1.0, min_gs_size=1,
                   overwrite = False, return_value='summary')

n_labels, n_genes = get_gvm_size(formatted_gvm_fname)
(n_labels, n_genes)

group = 'AE' # vanilla autoencoder

batch_size = 128
m = 1000 # middle dimension
l = 50 # latent dimension

model = build_vae(input_dim=n_genes, middle_dim = m, latent_dim = l, 
                batch_size=batch_size, optimizer='Adamax', lr=.001)
vae, enc, dec = (model['vae'], model['enc'], model['dec'])
vae.load_weights('./models/%s/weights/%04dm_%04dl.h5'%(group, m, l))     

z = enc.predict_generator(
GeneVec_Generator(formatted_gvm_fname, gvm_path='gvm', batch_size=1000, shuffle=False),
workers=4, use_multiprocessing=True, verbose=0)

euc_dist = pairwise_distances(z, metric='euclidean')
cos_sim = cosine_similarity(z)
labels = open_gvm(formatted_gvm_fname)['idx']

euc_dist_df = pd.DataFrame(euc_dist, index=labels, columns=labels)
cos_sim_df = pd.DataFrame(cos_sim, index=labels, columns=labels) 
euc_dist_df.iloc[:5, :5]
cos_sim_df.iloc[:5, :5]
euc_dist_df.to_pickle('./data/%s_DIST_EUC.pkl'%lib_name)
cos_sim_df.to_pickle('./data/%s_DIST_COS.pkl'%lib_name)
cos_sim_df2 = pd.read_pickle('./data/%s_DIST_COS.pkl'%lib_name)

np.all(cos_sim_df == cos_sim_df2)
# load data into new network instance and cluster
# net = Network(clustergrammer_widget)

#net.load_file('cars.tsv')
# net.load_df(cos_sim_df)
# # Z-score normalize the rows
# # net.normalize(axis='row', norm_type='zscore', keep_orig=True)

# # # filter for the top 100 columns based on their absolute value sum
# # net.filter_N_top('col', 100, 'sum')
# net.cluster()
# net.widget()

Significant genes for alcohol found in GWAS
       Signature  Number of Up Genes  Number of Down Genes
0      drug:2737                 225                   373
1      drug:2738                 479                   117
2      drug:3037                 331                   240
3      drug:3038                 299                   274
4      drug:3107                 315                   281
5      drug:3475                 298                   296
6      drug:3892                 279                   314
7      drug:3910                 300                   297
8     drug:DM190                 283                   317
9     drug:DM191                 308                   292
10    drug:DM192                 290                   310
11    drug:DM193                 352                   248
12    drug:DM194                 323                   277
13   drug:DM1637                 301                   299
14   drug:DM1638                 307                   293
15   drug:DM

True

In [4]:
# net = Network(clustergrammer_widget)

# load matrix file
net.load_df(cos_sim_df)
cos_sim_df.to_csv("alcohol_clustergrammer.tsv", sep="\t")
# cluster using default parameters
net.cluster()

# make interactive widget
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "CREEDS_drug:2737_down", "ini": 717, "clust": 421, "rank": 373,…

In [5]:
def clustergrammer_link_from_file(file):
    import time, requests
    resp = requests.post(
        'http://amp.pharm.mssm.edu/clustergrammer/matrix_upload/',
        files={'file': open(file, 'rb')},
    )
    if resp.status_code != 200:
        raise Exception('[ERROR]: clustergrammer ({}): {}'.format(
            resp.status_code,
            resp.text,
        ))
    # wait a tinybit before returning link (backoff)
    time.sleep(1)
    return resp.text

def display_link_inline(link):
    from IPython.display import HTML, IFrame, display
    display(HTML(
        '<a href="{}" target="_blank">{}</a>'.format(link, link)
    ))
    display(IFrame(link, width=1000, height=1000))
    
display_link_inline(
    clustergrammer_link_from_file("alcohol_clustergrammer.tsv"))